In [17]:
import numpy as np
import pandas as pd
import os
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

import sys
import time

In [18]:
# --- 1. Cấu hình ---
PROCESSED_DATA_FOLDER = 'processed_data'
MODEL_OUTPUT_FOLDER = 'models'

# Đường dẫn đến 4 tệp đã xử lý
X_TRAIN_FILE = os.path.join(PROCESSED_DATA_FOLDER, 'X_train.npy')
Y_TRAIN_FILE = os.path.join(PROCESSED_DATA_FOLDER, 'y_train.npy')
X_TEST_FILE = os.path.join(PROCESSED_DATA_FOLDER, 'X_test.npy')
Y_TEST_FILE = os.path.join(PROCESSED_DATA_FOLDER, 'y_test.npy')

# Tên tệp mô hình sẽ được lưu
MODEL_FILE = os.path.join(MODEL_OUTPUT_FOLDER, 'random_forest_model.pkl')

In [19]:
# --- 2. Tải Dữ liệu ---
print("Đang tải dữ liệu huấn luyện và kiểm tra...")

try:
    X_train = np.load(X_TRAIN_FILE)
    y_train = np.load(Y_TRAIN_FILE)
    X_test = np.load(X_TEST_FILE)
    y_test = np.load(Y_TEST_FILE)
    
    print("Tải dữ liệu thành công!")
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_test shape: {y_test.shape}")
    print("---------------------------------")
    print(f"Số ca gian lận trong tập Train: {np.sum(y_train)}")
    print(f"Số ca gian lận trong tập Test: {np.sum(y_test)}")
    
except FileNotFoundError as e:
    print(f"LỖI: Không tìm thấy tệp {e.filename}.")
    print("Vui lòng đảm bảo bạn đã chạy tệp tiền xử lý (preprocess) trước.")

Đang tải dữ liệu huấn luyện và kiểm tra...
Tải dữ liệu thành công!
X_train shape: (4453834, 10)
y_train shape: (4453834,)
X_test shape: (1908786, 10)
y_test shape: (1908786,)
---------------------------------
Số ca gian lận trong tập Train: 5749
Số ca gian lận trong tập Test: 2464


In [20]:
# --- 3. Áp dụng Kỹ thuật SMOTE ---
print("\n--- 3. Áp dụng Kỹ thuật SMOTE ---")
print("Bắt đầu xử lý mất cân bằng với SMOTE...")

smote = SMOTE(random_state=42)

print("Đang áp dụng SMOTE (có thể mất vài phút)...")
start_time = time.time()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
end_time = time.time()

print(f"Hoàn tất SMOTE sau {end_time - start_time:.2f} giây.")
print("---------------------------------")
print(f"Kích thước tập train ban đầu: {X_train.shape}")
print(f"Kích thước tập train sau SMOTE: {X_train_resampled.shape}")
print("\nKiểm tra sự cân bằng của tập Train (sau SMOTE):")
print(f"Số ca 'Bình thường' (0) sau SMOTE: {len(y_train_resampled) - np.sum(y_train_resampled)}")
print(f"Số ca 'Lừa đảo' (1) sau SMOTE:   {np.sum(y_train_resampled)}")



--- 3. Áp dụng Kỹ thuật SMOTE ---
Bắt đầu xử lý mất cân bằng với SMOTE...
Đang áp dụng SMOTE (có thể mất vài phút)...
Hoàn tất SMOTE sau 0.94 giây.
---------------------------------
Kích thước tập train ban đầu: (4453834, 10)
Kích thước tập train sau SMOTE: (8896170, 10)

Kiểm tra sự cân bằng của tập Train (sau SMOTE):
Số ca 'Bình thường' (0) sau SMOTE: 4448085
Số ca 'Lừa đảo' (1) sau SMOTE:   4448085


In [21]:
# --- 4. Xây dựng & Huấn luyện Mô hình RF (với SMOTE) ---
print("\n--- 4. Xây dựng & Huấn luyện Mô hình RF (với SMOTE) ---")
print("Bắt đầu huấn luyện mô hình Random Forest trên dữ liệu SMOTE...")

# Định nghĩa mô hình MỚI (không cần class_weight)
rf_model_smote = RandomForestClassifier(
    n_estimators=100,      
    random_state=42,       
    n_jobs=-1              
)

# Huấn luyện mô hình trên dữ liệu ĐÃ RESAMPLE
start_time = time.time()
rf_model_smote.fit(X_train_resampled, y_train_resampled)
end_time = time.time()
print(f"Huấn luyện (SMOTE) hoàn tất sau {end_time - start_time:.2f} giây.")


--- 4. Xây dựng & Huấn luyện Mô hình RF (với SMOTE) ---
Bắt đầu huấn luyện mô hình Random Forest trên dữ liệu SMOTE...
Huấn luyện (SMOTE) hoàn tất sau 389.19 giây.


In [22]:
# --- 5. Đánh giá Mô hình SMOTE (Sử dụng cấu trúc của bạn) ---
print("\n--- 5. ĐÁNH GIÁ MÔ HÌNH SMOTE TRÊN TẬP TEST ---")

# Dự đoán trên tập test (DÙNG MÔ HÌNH MỚI)
y_pred_smote = rf_model_smote.predict(X_test)

# In báo cáo phân loại
print("\nBáo cáo phân loại (Classification Report):")
report = classification_report(y_test, y_pred_smote, target_names=['Bình thường (0)', 'Lừa đảo (1)'])
print(report)

# In ma trận nhầm lẫn (Dùng cấu trúc DataFrame của bạn)
print("\nMa trận nhầm lẫn (Confusion Matrix):")
cm_smote = confusion_matrix(y_test, y_pred_smote)

# Dùng Pandas DataFrame để hiển thị đẹp hơn
cm_df = pd.DataFrame(cm_smote, 
                     index=['Đúng: Bình thường', 'Đúng: Lừa đảo'], 
                     columns=['Dự đoán: Bình thường', 'Dự đoán: Lừa đảo'])
print(cm_df)

# Phân tích chi tiết (giữ lại để dễ so sánh)
print("\nPhân tích chi tiết Ma trận nhầm lẫn (SMOTE):")
print(f"Bỏ lọt: Lừa đảo (FN):   {cm_smote[1, 0]}")
print(f"Báo nhầm: Lừa đảo (FP): {cm_smote[0, 1]}")
print(f"Đúng: Lừa đảo (TP):     {cm_smote[1, 1]}")
print(f"Đúng: Bình thường (TN): {cm_smote[0, 0]}")

print("\n--- Kết thúc ---")


--- 5. ĐÁNH GIÁ MÔ HÌNH SMOTE TRÊN TẬP TEST ---

Báo cáo phân loại (Classification Report):
                 precision    recall  f1-score   support

Bình thường (0)       1.00      0.97      0.99   1906322
    Lừa đảo (1)       0.04      0.92      0.08      2464

       accuracy                           0.97   1908786
      macro avg       0.52      0.95      0.53   1908786
   weighted avg       1.00      0.97      0.98   1908786


Ma trận nhầm lẫn (Confusion Matrix):
                   Dự đoán: Bình thường  Dự đoán: Lừa đảo
Đúng: Bình thường               1854317             52005
Đúng: Lừa đảo                       193              2271

Phân tích chi tiết Ma trận nhầm lẫn (SMOTE):
Bỏ lọt: Lừa đảo (FN):   193
Báo nhầm: Lừa đảo (FP): 52005
Đúng: Lừa đảo (TP):     2271
Đúng: Bình thường (TN): 1854317

--- Kết thúc ---
